In [ ]:
# Standard imports
import pandas as pd 
import numpy as np
import os
import sklearn
import scipy
from pathlib import Path

# Topic modeling with embeddings
from top2vec import Top2Vec
import joblib 

# Guided topic modeling
from corextopic import corextopic as ct
from sklearn.feature_extraction.text import CountVectorizer
import scipy.sparse as ss

# Sentiment analysis
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Interactive visuals
import streamlit as st
import plotly.express as px
import umap

# Path to JSON file (one row per comment; each row a dict/object)
DATA_PATH = Path("C:\\Users\\linna\\OneDrive\\Documents\\Python_Dev\\topic-modeling\\data\\public_comments.json")

df = pd.read_json(DATA_PATH, orient="records", lines=False)  # or lines=True if newline-delimited

# Ensure comments exist
TEXT_COL = "comment_text"     
if TEXT_COL not in df.columns:
    raise ValueError(f"{TEXT_COL} not found in dataframe columns: {df.columns.tolist()}")

df = df.dropna(subset=[TEXT_COL]).reset_index(drop=True)

print("Loaded", len(df), "comments")
df.head()

In [3]:
pr_238 = df[df['docket_id'] == 'TTB-2025-0003']
pr_237 = df[df['docket_id'] == 'TTB-2025-0002']

In [4]:
print('TTB-2025-0003: ' + str(len(pr_238)) + ' comments')
print('TTB-2025-0002: ' + str(len(pr_237)) + ' comments')

TTB-2025-0003: 189 comments
TTB-2025-0002: 174 comments


In [5]:
df = pr_238

In [6]:
df = df.dropna(subset=[TEXT_COL]).reset_index(drop=True)
print("Docs in df:", len(df))

Docs in df: 189


In [49]:
example_df = pd.concat([pr_238, pr_237])

In [63]:
example_df.to_csv("C:\\Users\\linna\\OneDrive\\Documents\\Python_Dev\\topic-modeling\\data\\example_data.csv", index=False)

In [7]:
# Prepare documents (list of comments as strings) and document (comment) ids
documents = df[TEXT_COL].astype(str).tolist()
# Keep comment_id as strings (Top2Vec will return these when using document_ids)
document_ids = df["comment_id"].astype(str).tolist()

# Set embedding model and training parameters
embedding_model = "all-MiniLM-L6-v2"
speed = "deep-learn"
workers = os.cpu_count() or 1

# Train model (may take a while depending on number of comments and chosen 'speed' 
model = Top2Vec(
    documents=documents,
    document_ids=document_ids,
    embedding_model=embedding_model,
    speed=speed,
    workers=workers
)

2025-08-28 14:10:45,572 - top2vec - INFO - Pre-processing documents for training
C:\Users\linna\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
2025-08-28 14:10:45,911 - top2vec - INFO - Downloading all-MiniLM-L6-v2 model
2025-08-28 14:10:47,505 - top2vec - INFO - Creating joint document/word embedding
2025-08-28 14:11:01,759 - top2vec - INFO - Creating lower dimension embedding of documents
2025-08-28 14:11:35,254 - top2vec - INFO - Finding dense areas of documents
C:\Users\linna\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\linna\anaconda3\Lib\site-packages\sklearn\utils\deprecation.py:132: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025-

In [8]:
# Topics
n_topics = model.get_num_topics()
print("Number of topics discovered:", n_topics)

topic_sizes, topic_nums = model.get_topic_sizes()
print("Top 10 topic sizes (docs per topic):")
for size, num in zip(topic_sizes[:10], topic_nums[:10]):
    print(f"  Topic {num}: {size} documents")

Number of topics discovered: 1
Top 10 topic sizes (docs per topic):
  Topic 0: 189 documents


In [9]:
# Top N words for top N topics
top_n = min(10, n_topics)
topic_words, word_scores, topic_numbers = model.get_topics(top_n)
for words, scores, tnum in zip(topic_words, word_scores, topic_numbers):
    print("\nTopic", tnum)
    # show top 10 words (they come ordered)
    for w, s in zip(words[:10], scores[:10]):
        print(f"  {w} ({s:.3f})")


Topic 0
  allergens (0.553)
  allergies (0.401)
  label (0.356)
  labeling (0.336)
  alcohol (0.335)
  allergen (0.315)
  beverages (0.315)
  labels (0.304)
  ingredients (0.303)
  alcoholic (0.262)


In [10]:
# Map dominant topic back to df
# Use search_documents_by_topic for each topic and assign docs to their best topic.
docid_to_topic = {}
topic_sizes, topic_nums = model.get_topic_sizes()  # topics ordered by size

for size, tnum in zip(topic_sizes, topic_nums):
    # retrieve all documents for this topic (num_docs = topic size)
    docs, doc_scores, doc_ids = model.search_documents_by_topic(topic_num=tnum, num_docs=int(size))
    # doc_ids are the original document_ids you passed to the model
    for did in doc_ids:
        # assign topic tnum as the dominant topic for did
        # if a doc appears in multiple lists (unlikely), first assignment wins (largest topic first)
        if did not in docid_to_topic:
            docid_to_topic[did] = tnum

# Create a column in df with the dominant topic (or -1 if not assigned)
df["top2vec_dominant_topic"] = df["comment_id"].astype(str).map(docid_to_topic).fillna(-1).astype(int)

In [11]:
# Label with top terms
top_n = 5

# Get the words for every topic the model discovered
num_topics = model.get_num_topics()
topic_words, word_scores, topic_nums = model.get_topics(num_topics)

topic_label_map = {}
for tnum, words in zip(topic_nums, topic_words):
    words_sel = words[:top_n]                     
    label = ", ".join(words_sel)                  
    topic_label_map[int(tnum)] = label

In [12]:
# map to df; fallback label for unknown topics
df["top2vec_terms"] = (
    df["top2vec_dominant_topic"]
      .map(topic_label_map)
      .fillna("Unclear")
)

In [13]:
# Save Topic2Vec model (Top2Vec has its own save/load)
model_save_path = "C:\\Users\\linna\\OneDrive\\Documents\\Python_Dev\\topic-modeling\\models\\top2vec_model"
model.save(model_save_path)
print("Model saved to:", model_save_path)

Model saved to: C:\Users\linna\OneDrive\Documents\Python_Dev\topic-modeling\models\top2vec_model
